# Rapidly generate figures for atlas and groups

To install the kernel used by NERSC-metatlas users, copy the following text to $HOME/.ipython/kernels/mass_spec_cori/kernel.json

```
{
 "argv": [
  "/global/common/software/m2650/python-cori/bin/python",
  "-m",
  "IPython.kernel",
  "-f",
  "{connection_file}"
 ],
 "env": {
    "PATH": "/global/common/software/m2650/python-cori/bin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin"
 },
 "display_name": "mass_spec_cori",
 "language": "python"
}
```


In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%matplotlib notebook
import sys
import os

#### add a path to your private code if not using production code ####
# sys.path.insert(0,"/global/homes/b/bpb/repos/metatlas/") #where your private code is
######################################################################

# metatlas imports
from metatlas.helpers import dill2plots as dp
from metatlas.helpers import fastanalysis as fa
import metatlas.metatlas_objects as metob

# other data tools you might need
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

# 1. Retrieve atlas by name

In [ ]:
atlases = metob.retrieve('Atlas',name='SK_EMA_MP-SolapP_POS_20170914-2108-proton',username='*')
names = []
for i,a in enumerate(atlases):
    print(i,a.name,pd.to_datetime(a.last_modified,unit='s'))#len(a.compound_identifications)

# 2. Pick which atlas from the list above. 

For example

```
atlas = atlases[0]
``` 

will select the first one

```
atlas = atlases[1] 
```

will select the second one... and so forth.

In [ ]:
atlas = atlases[0]
print atlas.name
print atlas.username

# 3. Specify groups and filter by include_list and exclude_list

In [ ]:
groups = dp.select_groups_for_analysis(name = '%20160818_SK-MP_SolarP_MeOHex1_QE119_ZICHIL-549760%',
                                       most_recent = True,
                                       remove_empty = True,
                                       include_list = [], exclude_list = ['NEG','QC','Bl'])#['QC','Blank'])

# 4. Replace 'results' with directory to store your results

In [ ]:
output_dir = 'results_fa'
output_dir = os.path.join(os.path.expanduser('~'),output_dir)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
print('your results will be written to %s'%output_dir)

# 5. Get data for atlas and groups and filter by mz, rt, and msms parameters

In [ ]:
fa = reload(fa)
original_dataset,filtered_dataset = fa.filter_and_dump(atlas, groups, output_dir,
                   min_intensity = 1e5,
                   rt_tolerance = 0.25,
                   mz_tolerance = 10,
                   min_msms_score = 0.2, allow_no_msms = False,
                   min_num_frag_matches = 1,  min_relative_frag_intensity = .01)

# 6. See how many compounds are remaining

In [ ]:
print('%d files and %d compounds in original dataset'%(len(original_dataset),len(original_dataset[0])))
print('%d files and %d compounds in filtered dataset'%(len(filtered_dataset),len(filtered_dataset[0])))